#### MVP: 
 * a streamlit dashboard and a tableau dashboard
 * filter by activities
 * **total time spent on activites by week**

#### ideas for this data: 
 * convert the map data to elevation data
 * something with the suffer score
   * suffer per second? suffer per minute? 

#### to do
 * convert start_date column to pandas timestamp 

In [50]:
import pandas as pd
#pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import streamlit as st
import time

import plotly.express as px
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

In [80]:
def get_strava_data(csv_file):
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    activities = pd.read_csv(csv_file)

    while True:
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
        r_df = pd.DataFrame.from_dict(r, orient='columns')
        print(r_df.head(5).id)        
               
        # if there are no new values (i.e. activities) found in r_df, just break.  otherwise keep going
        if len([v for v in list(r_df['id']) if v not in list(activities['id'])]) == 0:
            activities.drop_duplicates(subset=['id'], inplace=True)
            activities.sort_values(by='id', ascending=False, inplace=True)
            activities.to_csv(csv_file, index=False)
            return activities

        if (not r):   # if no results then exit loop
            break
            
        activities = pd.concat([r_df, activities])
        page += 1
    
    activities.drop_duplicates(subset=['id'], inplace=True)
    activities.sort_values(by='id', ascending=False, inplace=True)
    activities.to_csv(csv_file, index=False)
    return activities

In [74]:
new_acts = ['7413', '7412', '7411', '7410']
old_acts = ['7413', '7412', '7411', '7410']

In [75]:
len([v for v in new_acts if v not in old_acts]) == 0

True

In [82]:
#get_strava_data('strava_activities.csv')

In [227]:
# always remember to pass index=False to not add an Unnamed column on your export! 


In [120]:
strava_activities_clean.loc[strava_activities_clean.id == 7299973019, 'start_time']

17   2022-06-13 02:08:36+00:00
Name: start_time, dtype: datetime64[ns, UTC]

In [104]:
cols_to_drop = ['Unnamed: 0.1', 'athlete', 'resource_state', 'start_date_local', 'timezone', 
'utc_offset', 'trainer', 'commute', 'type', 'manual', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'display_hide_heartrate_option',
'upload_id_str', 'external_id', 'from_accepted_tag', 'total_photo_count', 'athlete.resource_state', 'map.id', 'map.summary_polyline', 'map.resource_state', 
'average_watts', 'kilojoules', 'device_watts', 'photo_count', 'heartrate_opt_out', 'upload_id', 'athlete.id', 'workout_type',
'has_heartrate', 'location_city', 'location_state', 'location_country', 'private', 'has_kudoed', 'Unnamed: 0']

strava_activities_raw = pd.read_csv('strava_activities.csv')

strava_activities_clean = strava_activities_raw.drop(labels=cols_to_drop, axis=1)

# meters to miles
strava_activities_clean['distance'] = strava_activities_clean.distance * .000621

# might be a fun metric to plot over time. perhaps my suffering per minute has gone down as ive gotten fitter
strava_activities_clean['SPM'] = strava_activities_clean.suffer_score / (strava_activities_clean.moving_time / 60)

rename = {'start_date': 'start_time'}
strava_activities_clean.rename(rename, axis=1, inplace=True)

# convert start_time to pandas Timestamp object. then we can access seconds, minutes, hours, etc. and can add the moving_time
strava_activities_clean['start_time'] = pd.to_datetime(strava_activities_clean['start_time'])

strava_activities_clean.loc[strava_activities_clean.id == 7299973019, 'start_time'] = strava_activities_clean.loc[strava_activities_clean.id == 7299973019, 'start_time'].replace(day=12)

strava_activities_clean.set_index('start_time', inplace=True)

TypeError: Series.replace() got an unexpected keyword argument 'day'

In [94]:
strava_activities_clean.iloc[17]

name                                                          Evening Run
distance                                                         6.158146
moving_time                                                          3467
elapsed_time                                                         3499
total_elevation_gain                                                233.0
sport_type                                                            Run
id                                                             7299973019
achievement_count                                                       6
kudos_count                                                             5
comment_count                                                           0
athlete_count                                                           1
map                     {'id': 'a7299973019', 'summary_polyline': 'q{z...
average_speed                                                        2.86
max_speed                             

In [95]:
tst = strava_activities_clean.index[17]

In [98]:
strava_activities_clean.index[17] = strava_activities_clean.index[17].replace(day=12)

TypeError: Index does not support mutable operations

In [88]:
# my 6/12 run is showing up as 6/13
strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(15)

start_time
2022-04-04 00:00:00+00:00    30.598409
2022-04-11 00:00:00+00:00    32.343108
2022-04-18 00:00:00+00:00    39.471629
2022-04-25 00:00:00+00:00    22.761140
2022-05-02 00:00:00+00:00    41.244522
2022-05-09 00:00:00+00:00    26.253520
2022-05-16 00:00:00+00:00    20.777232
2022-05-23 00:00:00+00:00    43.479315
2022-05-30 00:00:00+00:00    30.362615
2022-06-06 00:00:00+00:00    33.258152
2022-06-13 00:00:00+00:00    27.664060
2022-06-20 00:00:00+00:00    28.054979
2022-06-27 00:00:00+00:00    42.420821
2022-07-04 00:00:00+00:00    26.436405
2022-07-11 00:00:00+00:00     5.492310
Freq: W-MON, Name: distance, dtype: float64

In [97]:
#strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].head(30)


In [42]:
strava_activities_clean.head(3)

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,max_speed,average_cadence,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-06-30 21:39:30+00:00,Afternoon Run,6.014199,3682,3685,314.0,Run,7395188303,11,3,0,...,4.392,84.4,28.0,154.1,180.0,183.2,48.2,6,102.0,1.662140
2022-06-30 00:12:17+00:00,Afternoon Golf,6.437597,10586,14280,120.0,Golf,7390944431,0,2,0,...,2.624,NaN,28.0,102.5,148.0,93.8,68.4,0,29.0,0.164368
2022-06-29 19:24:38+00:00,Lunch Run,4.054633,2165,2168,132.0,Run,7389543804,3,1,0,...,4.302,87.0,27.0,150.8,167.0,181.4,49.0,0,47.0,1.302540


In [106]:
x = 'start_time'
y = 'average_speed'
df = strava_activities_clean[(strava_activities_clean.sport_type == 'Run') & (strava_activities_clean.distance.between(3.99, 8.5))]

fig = px.scatter(data_frame=df, 
                        y=y,   # choice 
                        x=x,
                        #hover_data=['distance', 'calories'], 
                        #labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title=str(y + ' x ' + x),
                        #size='distance',
                        #color='distance',
                        template='plotly',
                        height=500, 
                        width=800, 
                        #trendline='ols'
                        )

In [107]:
fig.show()

In [76]:
type(strava_activities_clean['start_time'][2])

str

In [66]:
pd_dt_obj

Timestamp('2022-06-28 19:05:39+0000', tz='UTC')

In [65]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

Timestamp('2022-06-28 20:05:39+0000', tz='UTC')

In [67]:
regular_dt_obj

numpy.datetime64('2022-06-28T19:05:39.000000000')